In [11]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough #input이 다음 체인으로 넘어가게 해줌
from langchain_core.runnables import RunnableLambda

#랭스미스와의 연동을 위해 필요 (2024년 말 업데이트 이후 자동 트레이싱이 비활성화 됨. 수동으로 callbacks를 줘야 함)
from langsmith import Client
from langchain.callbacks.tracers.langchain import LangChainTracer

from langchain_openai import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.vectorstores import Chroma
from langchain.memory import ConversationBufferMemory

client = Client()
tracer = LangChainTracer(client=client)
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.1, callbacks=[tracer])

cache_dir = LocalFileStore("./.cache/") 

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100
)

loader = UnstructuredFileLoader("../files/chapter_three.txt")

docs=loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

vectorstore = Chroma.from_documents(docs, cached_embeddings)

results = vectorstore.similarity_search("Where does Winston live?")

retriever = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n{context}"),
    MessagesPlaceholder(variable_name="history"),
    ("human","{question}")
])

memory = ConversationBufferMemory(return_messages=True)

def load_memory(_):
    print(_)
    return memory.load_memory_variables({})["history"]

chain = (RunnablePassthrough.assign(history=load_memory) 
         | {"context":RunnableLambda(lambda x: retriever.invoke(x["question"])), 
            "question":RunnablePassthrough(),
            "history":RunnableLambda(lambda x: x["history"])} 
         | prompt | llm
)

def invoke_chain(question):
    result = chain.invoke({"question":question})
    memory.save_context(
        {"input":question}, 
        {"output":result.content}
    )
    print(result)

invoke_chain("Is Aaronson guilty?")
invoke_chain("What message did he write in the table?")
invoke_chain("Who is Julia?")

libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.


{'question': 'Is Aaronson guilty?'}
content="I don't have that information." additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 2269, 'total_tokens': 2277, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BTAaOpOpex0I437oyotr8wRI7Onyt', 'finish_reason': 'stop', 'logprobs': None} id='run-71271ed2-5533-4572-8f77-f26bde1050b5-0' usage_metadata={'input_tokens': 2269, 'output_tokens': 8, 'total_tokens': 2277, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
{'question': 'What message did he write in the table?'}
content='He wrote "FREEDOM IS SLAVERY" and "TWO AND TWO MAKE FIVE" in the table.' additional_kwargs={'refusal': None} response_met